In [1]:
import pandas as pd
import os
import docx

In [2]:
import ast
import javalang
dataset =[]
def is_parsable(text, ext, fp):
    if text != -1:
        if ext=='py':
            try:
                _ = ast.parse(text)
                dataset.append([text, ext, 1, fp])
                return True
            except Exception as error:
                print("for python ", type(error).__name__)
                dataset.append([text, ext, 0, fp])
                return False
        elif ext=='java':
            try:
                programtokens = javalang.tokenizer.tokenize(text)
                parser = javalang.parse.Parser(programtokens)
                _ = parser.parse_member_declaration()
                dataset.append([text, ext, 1, fp])
                return True
            except:
                try:
                    programtokens = javalang.tokenizer.tokenize(text)
                    _ = javalang.parser.parse(programtokens)
                    dataset.append([ext, ext,1, fp])
                    return True
                except Exception as error:
                    print("for java:", type(error).__name__)
                    dataset.append([text, ext, 0, fp])
                    return False

In [3]:
def save_as_json(df, file_name, output_path):
    output_path = output_path+file_name
    with open(output_path, "w") as f:
        f.write(df.to_json(orient='records', lines=True))
    return True

In [4]:
def get_all_files(target_dir):
   all_files = []
   for root, dirs, files in os.walk(target_dir, topdown=False):
      for name in files:
         s = os.path.join(root, name)
         
         all_files.append(s)
   return all_files


In [5]:
gcb_path = "/home/egk204/projects/clone-type-iv/GptCloneBench-main/standalone"

def split_data(data, rt):
    data_num = len(data)
    ratios = [8,1,1]
    
    train_split = int(ratios[0]/sum(ratios)*data_num)
    val_split = train_split + int(ratios[1]/sum(ratios)*data_num)

    data = data.sample(frac=1, random_state=12345)

    train = data.iloc[:train_split]
    val = data.iloc[train_split:val_split]
    test = data.iloc[val_split:]
    train.to_csv(f"{rt}/train.csv", index=False, header=None)
    val.to_csv(f"{rt}/valid.csv", index=False, header=None)
    test.to_csv(f"{rt}/test.csv", index=False, header=None)
    return train, val, test

### single language

In [6]:
import re
def split_java_functions_from_file(file_path):
    with open(file_path, "r") as file:
        file_content = file.read()
    split_texts = re.split(r'\n{2,}', file_content)
    if len(split_texts) >= 2:
        c1_lines = split_texts[0].split('\n')
        if len(c1_lines) > 0:
            c1_lines[0] = c1_lines[0].replace('#input', '')
        c1 = '\n'.join(c1_lines)
        c2 = "".join(split_texts[1:])
    return c1, c2

In [7]:
def get_key(key_map, k):
    if k not in key_map:
        key_map[k]=len(key_map)
        return key_map[k]
def get_single_code_dict_and_pairs(all_files):
    codes = []
    pairs = []
    parsable_codes = []
    parsable_pairs = []
    key_map ={}
    for file_path in all_files:
        k = "_".join(file_path.split("/")[6:]).split(".")[0]
        c1, c2 = split_java_functions_from_file(file_path)  
        tk1 = k+"_1"    
        tk2 = k+"_2"
        
        k1 = get_key(key_map, tk1)
        k2 = get_key(key_map, tk2)
        
        codes.append([c1, k1])
        codes.append([c2, k2])
        pairs.append([k1, k2]+[1 if "true" in k else 0])

        f1 = is_parsable(c1, 'java', k1)
        if f1:
            parsable_codes.append([c1, k1])
        else:
            print(file_path, "\nc1:\n", c1)
        

        f2 = is_parsable(c2, 'java', k2)
        if f2:
            parsable_codes.append([c2, k2])
        else:
            print(file_path,"\nc2:\n", c2)

        if f1 and f2:
            parsable_pairs.append([k1, k2]+[1 if "true" in k else 0])

    return codes, pairs, parsable_codes, parsable_pairs, key_map
    # return pd.DataFrame.from_dict(code_dict, orient='index'), pd.DataFrame(pair_info)

In [8]:
single_true_java_files = get_all_files(f"{gcb_path}/true_semantic_clones/java/")
single_false_java_files = get_all_files(f"{gcb_path}/false_semantic_clones/java/")
all_files = single_true_java_files+single_false_java_files

In [9]:
print(len(all_files))

11165


In [10]:
codes, pairs, parsable_codes, parsable_pairs, key_map = get_single_code_dict_and_pairs(all_files)

for java: JavaSyntaxError
/home/egk204/projects/clone-type-iv/GptCloneBench-main/standalone/true_semantic_clones/java/prompt_2/Type_3/Gpt3D_Clone368_4.java 
c2:
  public static void main (String [] args) {
    int sayThisManyTimes = 5;
    volatile boolean saidHello = false;
    MyThread mt = new MyThread (sayThisManyTimes, saidHello);
    mt.start ();
    for (int i = 0;
    i < sayThisManyTimes; i ++) {
        while (saidHello) {
        }
        System.out.println ("Hello ");
        saidHello = true;
    }
}
for java: JavaSyntaxError
/home/egk204/projects/clone-type-iv/GptCloneBench-main/standalone/true_semantic_clones/java/prompt_2/Type_3/Gpt3D_Clone85_4.java 
c2:
  public void start (Stage primaryStage) {
    final HTMLEditor htmlEditor = new HTMLEditor ();
    primaryStage.setScene (new Scene (htmlEditor));
    primaryStage.show ();
    Node toolBar = htmlEditor.lookup (".tool-bar");
    while (toolBar != null) {
      ((Pane) toolBar.getParent ()).getChildren ().remove (toolB

In [11]:
import pickle
def dump_pickle(file_to_dump, fp):
    with open(fp, 'wb') as p:
        pickle.dump(file_to_dump, p)
        return "saved as: "+fp

In [12]:
output_path = "/home/egk204/projects/clone-type-iv/data/gptcb/" 
dump_pickle(key_map, output_path+"single_key_map.pickle")

'saved as: /home/egk204/projects/clone-type-iv/data/gptcb/single_key_map.pickle'

In [13]:

# only parsable
codes_single_df = pd.DataFrame(parsable_codes, columns=["func", "idx"])
save_as_json(codes_single_df, "data.jsonl", output_path)
df_pairs = pd.DataFrame(parsable_pairs)
df_pairs.to_csv(output_path+"java_single_pairs.csv", index=False)
split_data(df_pairs, output_path)

# saving all
codes_single_df_all = pd.DataFrame(codes, columns=["func", "idx"])
save_as_json(codes_single_df, "data_including_unparsable.jsonl", output_path)
df_pairs = pd.DataFrame(pairs)
df_pairs.to_csv(output_path+"java_single_pairs_including_unparsable.csv", index=False, header=None)
# split_data(df_pairs, output_path)

# code_dict, pair_info, 

In [14]:
df_pairs

,0,1,2
0,0,1,1
1,2,3,1
2,4,5,1
3,6,7,1
4,8,9,1
...,...,...,...
11160,22320,22321,0
11161,22322,22323,0
11162,22324,22325,0
11163,22326,22327,0


In [15]:
len(parsable_pairs), len(pairs)

(11141, 11165)

In [16]:
dataset=[]
for i in codes: #is_parsable(text, ext, fp):
    is_parsable(i[0], "java", i[1])
parsability_checked = pd.DataFrame(dataset, columns=["func", "lang", "is_parsable", "idx"])
parsable = parsability_checked[parsability_checked.is_parsable==1]
unparsable = parsability_checked[parsability_checked.is_parsable==0]

for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError
for java: JavaSyntaxError


In [17]:
unparsable.shape, 

((24, 4),)

In [18]:
unparsable

,func,lang,is_parsable,idx
2239,public static void main (String [] args) {\n ...,java,0,2239
2287,public void start (Stage primaryStage) {\n ...,java,0,2287
6897,"public static void close (Statement...stmts, ...",java,0,6897
7075,public static void close (Connection...connec...,java,0,7075
7397,public static void main(String... args) throw...,java,0,7397
8129,public void run() {\n volatile boolean sai...,java,0,8129
8365,public String[][] allUniqueCombinations() {\n...,java,0,8365
8369,public FileVisitResult postVisitDirectory (Pa...,java,0,8369
8933,public String getNextMonth (boolean maxDate) ...,java,0,8933
9329,"private static String join(String delimiter,...",java,0,9329


In [19]:
print(unparsable.iloc[0,:].idx,"\n" ,unparsable.iloc[0,:].func)

2239 
  public static void main (String [] args) {
    int sayThisManyTimes = 5;
    volatile boolean saidHello = false;
    MyThread mt = new MyThread (sayThisManyTimes, saidHello);
    mt.start ();
    for (int i = 0;
    i < sayThisManyTimes; i ++) {
        while (saidHello) {
        }
        System.out.println ("Hello ");
        saidHello = true;
    }
}


In [20]:
print(unparsable.iloc[5,:].idx,"\n" ,unparsable.iloc[5,:].func)

8129 
  public void run() {
    volatile boolean saidHello = false;
    for (int i=0; i < sayThisManyTimes; i++) {
        while (! saidHello) {
            System.out.println("Hello");
            saidHello = true;
        }
        System.out.println("World!");
        saidHello = false;
    }
}


In [21]:
# func = ext.split("def")
# "def"+f.strip() for f in func[1:] if func.strip()

### cross language data

In [22]:
def splitter(file_content):
    if "def " in file_content:
        func = file_content.split("def ")
        c1 = func[0]
        c1_lines = c1.split('\n')
        if len(c1_lines) > 0:
            c1_lines[0] = c1_lines[0].replace('#input', '')
        c1 = '\n'.join(c1_lines)
        
        # print(f"total {len(func)} c1:\n",c1)
        # print("------")
        c2 = "def " + func[1].strip()
        # print(f"{len(func)-1} c2:\n",c2)
        if len(func)>2:
            tmp = "\n".join(["def " + f.strip() for f in func[2:]])
            c2=c2+"\n"+tmp
            c2=c2.strip()
            print(f"{len(func)-1} c2:\n",c2)
    # else:
    #     return "", ""
    else:
        split_texts = re.split(r'\n{2,}', file_content)
        if len(split_texts) >= 2: #it must be >=2
            c1_lines = split_texts[0].split('\n')
            if len(c1_lines) > 0:
                c1_lines[0] = c1_lines[0].replace('#input', '')
            c1 = '\n'.join(c1_lines)
            c2 = "\n".join(split_texts[1:])

    return c1, c2

In [23]:
def get_key(key_map, k):
    if k not in key_map:
        key_map[k]=len(key_map)
        return key_map[k]
    
def split_cross_funcs(all_cross_java):
    codes =[]
    parsable_codes=[]
    pairs = []
    parsable_pairs = []
    key_map ={}
    count_py = 0
    for i in all_cross_java:
        if "py" in i:
            count_py+=1
            k = "_".join(i.split("/")[6:]).split(".")[0]   

            doc = docx.Document(i)
            file_content = '\n'.join([paragraph.text for paragraph in doc.paragraphs])
            
            c1, c2 = splitter(file_content)
            
            tk1 = k+"_1"    
            tk2 = k+"_2"
            
            k1 = get_key(key_map, tk1)
            k2 = get_key(key_map, tk2)

            codes.append([k1, c1, "java"])
            codes.append([k2, c2, "py"])
            pairs.append([k1, k2, 1])
            
            if is_parsable(c1, 'java', k1) and is_parsable(c2, 'py', k2):
                parsable_codes.append([ k1, c1,"java"])
                parsable_codes.append([k2, c2,  "py"])
                parsable_pairs.append([k1, k2, 1])
            # else:
                # print(i)
                # print(file_content)
        else:
            continue
    # print(count_py)
    return codes, pairs, parsable_codes, parsable_pairs, count_py, key_map
dataset =[]        
all_cross_java = get_all_files(f"{gcb_path}/true_semantic_clones/cross_language/java/")
codes, pairs, parsable_codes, parsable_pairs, count_py, key_map = split_cross_funcs(all_cross_java)

for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for python  IndentationError
2 c2:
 def sequenceNums (nums):
    if not nums:
        return ""
    begin = nums[0]
    end = nums[0]
    sb = []
    for num in nums:
        if num - end <= 1:
            end = num
        else:
            appendRange(sb, begin, end)
            begin = end = num
        appendRange(sb, begin, end)
    return ''.join(sb)[1:]
def appendRange(sb, begin, end):
    sb.append(",{}-{}".format(begin, end) if begin != end else ",{}".format(begin))
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
3 c2:
 def test_method_call(object,loops):
    for _ in range(loops):
        s = object.getClass()
regular_calls = time.time_ns() - start
import inspect
method_ = inspect.getmembers(object, predicate=inspect.ismethod)
start = time.time_ns()
def test_invoke(object,loops,method):
    for _ in range(loops):
        s = method(object)
reflective_calls = time.time_ns() - start


for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
2 c2:
 def findMinMax(arr):
    max = - float('inf')
    min = float('inf')
    for num in arr:
        if num > max:
            max = num
        elif num < min:
            min = num
    return (min, max)
def main():
    arr_len = int(input("Enter the number of elements: "))
    arr = [int(input("Enter the elements:")) for i in range(arr_len)]
    min, max = findMinMax(arr)
    print("Smallest Value in array:", min)
    print("Highest Value in array:", max)

if __name__ == "__main__":
    main()
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for java: LexerError
for python  SyntaxError
for python  IndentationError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for java: LexerError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for java: LexerError
2 c2:
 def solution(S, P, Q): 
    N = len(S

In [24]:
len(pairs), len(parsable_pairs)

(964, 888)

pass = "admin"

test = input("Insert the admin pass: ")

if pass == test:

    print("Access granted")

else:

    for i in range(1, 3):

        test = input(f"You have {i} try(es): ")

if pass == test:

    print("Access granted")

    break

elif i == 2:

    print("Access denied")

In [25]:
pd.DataFrame(codes, columns=["idx", "func", "lang"])

,idx,func,lang
0,0,\n public void onClick (View v) {\n ...,java
1,1,def onClick (view):\n row_index = position\...,py
2,2,\npublic void actionPerformed (ActionEvent ae...,java
3,3,def actionPerformed(ae):\n if ae.getSource(...,py
4,4,\npublic synchronized void run () {\n int ...,java
...,...,...,...
1923,1923,def countLines(str):\n if not str or not st...,py
1924,1924,\npublic static void main (String [] args) {\...,java
1925,1925,def main(args):\n length = len(args)\n i...,py
1926,1926,\npublic static void main (String [] args) {\...,java


In [26]:
output_path = "/home/egk204/projects/clone-type-iv/data/gptcb_cross/" 
dump_pickle(key_map, output_path+"cross_key_map.pickle")

'saved as: /home/egk204/projects/clone-type-iv/data/gptcb_cross/cross_key_map.pickle'

In [27]:


cross_codes_df_parsable = pd.DataFrame(parsable_codes, columns=["idx", "func", "lang"])
save_as_json(cross_codes_df_parsable, "data.jsonl", output_path)
pair_cross_df_parsable = pd.DataFrame(parsable_pairs)
pair_cross_df_parsable.to_csv(output_path+"java_python_cross_pairs.csv", index=False, header=None)
split_data(pair_cross_df_parsable, output_path)

#all
cross_codes_df = pd.DataFrame(codes, columns=["idx", "func", "lang"])
save_as_json(cross_codes_df, "data_including_unparsable.jsonl", output_path)
pair_cross_df = pd.DataFrame(pairs)
pair_cross_df.to_csv(output_path+"java_python_cross_pairs_including_unparsable.csv", index=False, header=None)
# split_data(pair_cross_df, output_path)



In [28]:
dataset=[]
for i in codes:
    is_parsable(i[1], i[2], i[0]) #is_parsable(v, 'java', k)

parsability_checked = pd.DataFrame(dataset, columns=["func", "lang", "is_parsable", "idx"])
parsable = parsability_checked[parsability_checked.is_parsable==1]
unparsable = parsability_checked[parsability_checked.is_parsable==0]

for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for python  IndentationError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for java: LexerError
for python  SyntaxError
for python  IndentationError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for java: LexerError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for java: LexerError
for python  SyntaxError
for java: LexerError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for python  IndentationError
for python  SyntaxError
for python  SyntaxError
for python  IndentationError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for python  SyntaxError
for pyth

In [29]:
for i, r in unparsable.iterrows():
    print(r.func)

def run(s): 
    i = 2
    while i < 50:
        while s.getState() != 1: 
            with s lock: 
                try: 
                    s.wait() 
                except InterruptedException as e: 
                    print(e)
                    
        with s lock:
            if s.getState() == 1: 
                print("t2", i) 
            s.setState(2)
            i = i + 3
            s.notify_all()
def next(self):
    if not self.has_next:
        raise StopIteration 
    output = [self.arr[index] for index in self.ind]
    self.has_next = False
    for tail in range(len(self.ind) - 1, 0, -1):
        if self.ind[tail-1] < self.ind[tail):
            s = len(self.ind)-1
            while self.ind[tail-1] >= self.ind[s]:
                s -= 1
            self.swap(ind, tail-1, s)
            for i in range(tail, len(self.ind)-1):
                j = len(self.ind) - 1 - (i - tail)
                self.swap(ind, i,j)
            self.has_next = True
            break
    r

def run(s): 
    i = 2
    while i < 50:
        while s.getState() != 1: 
            with s lock: 
                try: 
                    s.wait() 
                except InterruptedException as e: 
                    print(e)
                    
        with s lock:
            if s.getState() == 1: 
                print("t2", i) 
            s.setState(2)
            i = i + 3
            s.notify_all()

def actionPerformed (ae):
if count < len(charsTiming):
    highlightNextWord ()
    ae.getSource().setInitialDelay(charsTiming[count])
    ae.getSource().restart ()
else:
    reset()

In [30]:
print(unparsable.iloc[1, :].func, unparsable.iloc[1, :].idx)

def next(self):
    if not self.has_next:
        raise StopIteration 
    output = [self.arr[index] for index in self.ind]
    self.has_next = False
    for tail in range(len(self.ind) - 1, 0, -1):
        if self.ind[tail-1] < self.ind[tail):
            s = len(self.ind)-1
            while self.ind[tail-1] >= self.ind[s]:
                s -= 1
            self.swap(ind, tail-1, s)
            for i in range(tail, len(self.ind)-1):
                j = len(self.ind) - 1 - (i - tail)
                self.swap(ind, i,j)
            self.has_next = True
            break
    return output 9


def next(self):
    if not self.has_next:
        raise StopIteration 
    output = [self.arr[index] for index in self.ind]
    self.has_next = False
    for tail in range(len(self.ind) - 1, 0, -1):
        if self.ind[tail-1] < self.ind[tail): #<<error here
            s = len(self.ind)-1
            while self.ind[tail-1] >= self.ind[s]:
                s -= 1
            self.swap(ind, tail-1, s)
            for i in range(tail, len(self.ind)-1):
                j = len(self.ind) - 1 - (i - tail)
                self.swap(ind, i,j)
            self.has_next = True
            break
    return output

def next(self): # copied this from the docx

    if not self.has_next:

        raise StopIteration

    output = [self.arr[index] for index in self.ind]

    self.has_next = False

    for tail in range(len(self.ind) - 1, 0, -1):

        if self.ind[tail-1] < self.ind[tail):

            s = len(self.ind)-1

    while self.ind[tail-1] >= self.ind[s]:

        s -= 1

        self.swap(ind, tail-1, s)

    for i in range(tail, len(self.ind)-1):

        j = len(self.ind) - 1 - (i - tail)

        self.swap(ind, i,j)

        self.has_next = True

        break

    return output

In [31]:
print(unparsable.iloc[25, :].func, unparsable.iloc[25, :].idx)

def main():    
    numberofDays = int(input("Number of days in the period: "))
    sharePoints = int(input("Share points on the first day: "))
    numberofDays = validator(numberofDays)
    outPutTablePrinter(numberofDays, sharePoints)

 #C
int main () 
{ 
    int numberofDays;
    int sharePoints;
    printf("Number of days in the period: ");
    scanf("%d", &numberofDays);
    printf("Share points on the first day: ");
    scanf("%d", &sharePoints);
    numberofDays = validator (numberofDays);
    outPutTablePrinter (numberofDays, sharePoints);
    return 0; 
} 401


In [32]:
unparsable

,func,lang,is_parsable,idx
5,def run(s): \n i = 2\n while i < 50:\n ...,py,0,5
9,def next(self):\n if not self.has_next:\n ...,py,0,9
21,def actionPerformed(e):\n duration = System...,py,0,21
49,def actionPerformed (ae):\nif count < len(char...,py,0,49
87,"def is_between(current_month, current_day, fro...",py,0,87
...,...,...,...,...
1820,\nprivate static int solution (int N) {\n ...,java,0,1820
1823,"def solution(X, A): \n n = len(A) \n\t\n\t# i...",py,0,1823
1831,def main(args): \n r = lambda: (\n m...,py,0,1831
1843,"def main(args): \n input = ""(A,Name1,200), ...",py,0,1843
